This solution takes O(P+D) time where P is the number of projects and D the number of dependency pairs.

In [1]:
class Project:
    def __init__(self, name):
        self.name = name
        self.children = {}
        self.dependencies = 0
        
    def addNeighbour(self, node):
        if self.children.get(node.getName()) is None:
            self.children[node.getName()] = node
            node.incrementDependencies()
            
    def getName(self):
        return self.name
    
    def getChildren(self):
        return [self.children[key] for key in self.children.keys()]
    
    def getNumberDependencies(self):
        return self.dependencies
    
    def incrementDependencies(self):
        self.dependencies += 1
        
    def decrementDependencies(self):
        self.dependencies -= 1
        
class Graph:
    def __init__(self):
        self.nodes = {}
        
    def getOrCreateNode(self, name):
        if self.nodes.get(name) is None:
            node = Project(name)
            self.nodes[name] = node

        return self.nodes.get(name)
        
    def addEdge(self, startName, endName):
        start = self.getOrCreateNode(startName)
        end = self.getOrCreateNode(endName)
        
        start.addNeighbour(end)
        
    def getNodes(self):
        return [self.nodes[key] for key in self.nodes.keys()]

In [2]:
def buildGraph(projects, dependencies):
    graph = Graph()
    
    # First, create all the projects and then add directed links for all the dependencies
    for project in projects:
        graph.getOrCreateNode(project)
        
    for dependency, project in dependencies:
        graph.addEdge(dependency, project)
        
    return graph

### Solution to problem

In [3]:
def findBuildOrder(projects, dependencies):
    graph = buildGraph(projects, dependencies)
    return orderProjects(graph.getNodes())

In [4]:
def addNonDependent(order, projects, offset):
    """ Returns a list with projects with zero dependencies """
    for project in projects:
        if project.getNumberDependencies() == 0:
            order[offset] = project
            offset += 1
    return offset, order

In [5]:
def orderProjects(projects):
    """ Returns a list of the projects in a correct build order """
    order = [None] * len(projects)

    # Add roots to the build order
    endOfList, order = addNonDependent(order, projects, 0)
    
    toBeProcessed = 0
    while toBeProcessed < len(order):
        current = order[toBeProcessed]
       
        # If the current project is None meaning there are no projects without dependencies, we have circular dependency
        if current is None:
            return None
        
        # Remove myself as a dependency
        children = current.getChildren()
        for child in children:
            child.decrementDependencies()
           
        endOfList, order = addNonDependent(order, children, endOfList)
        toBeProcessed += 1
       
    return order

In [6]:
projects = ["f","c","b","a","e","d","g"]
dependencies = [["f","c"],["f","a"],["f","b"],["c","a"],["b","a"],["b","e"],["a","e"],["d","g"]]

In [7]:
order = findBuildOrder(projects, dependencies)


In [9]:
for o in order:
    print(o.name)

f
d
c
b
g
a
e
